### 재료 2 - 군집화 안 좋음

In [1]:
from sklearn.cluster import KMeans # KMeans 라이브러리 불러오기
import numpy as np
import pandas as pd
import math
import seaborn as sb # 데이터 시각화를 위한 seaborn
import matplotlib.pyplot as plt # 데이터 시각화를 위한 matplotlib
import scipy as sp
from scipy.spatial import distance
sb.set_palette("Set2")

In [2]:
from google.cloud import bigquery
import pandas as pd

# BigQuery 클라이언트 초기화
client = bigquery.Client(project='jnu-idv-21')

# 쿼리 작성
query = """
SELECT *
FROM `0808.baby_food_tbl2`
"""

# 쿼리 실행 및 결과를 Pandas DataFrame으로 변환
df = client.query(query).to_dataframe()
df

,food_name,ingredient_name,ingredient_index,ingredient_gram,calorie,carbohydrate,protein,fat,fiber,calcium,phosphorus,iron,sodium,potassium,vitamin_A,thiamine,riboflavin,niacin,vitamin_C
0,찹쌀미음,"[쌀, 찹쌀, 백미]","[0, 18, 19]","[-1, -1, 23]",86.000000,18.700001,1.7,0.1,0.1,0.900000,34.700001,0.5,0.700000,43.900002,0.000000,0.0,0.0,0.4,0.0
1,닭가슴살고구마죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 닭고기, 가슴살, 날것, 고구마...","[0, 1, 2, 3, 4, 5, 27, 62, 28, 8, 7]","[-1, -1, -1, -1, -1, 20, -1, -1, 20, -1, 20]",117.000000,21.400000,6.2,0.4,0.7,9.800000,63.400002,0.5,16.400000,157.199997,4.800000,0.0,0.0,2.7,5.2
2,닭가슴살당근죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 닭고기, 가슴살, 날것, 당근,...","[0, 1, 2, 3, 4, 5, 27, 62, 28, 47, 7]","[-1, -1, -1, -1, -1, 20, -1, -1, 20, -1, 20]",98.199997,16.900000,6.1,0.4,0.8,12.600000,60.000000,0.5,19.200001,143.800003,252.399994,0.0,0.0,2.7,1.4
3,닭가슴살청경채죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 닭고기, 가슴살, 날것, 청경채...","[0, 1, 2, 3, 4, 5, 27, 62, 28, 39]","[-1, -1, -1, -1, -1, 20, -1, -1, 20, 15]",93.500000,15.700000,6.1,0.4,0.7,18.500000,58.299999,0.5,15.600000,105.400002,52.799999,0.0,0.0,2.6,7.4
4,찹쌀컬리플라워당근죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 쌀, 찹쌀, 백미, 컬리플라워,...","[0, 1, 2, 3, 4, 5, 0, 18, 19, 67, 47, 7]","[-1, -1, -1, -1, -1, 20, -1, -1, 5, 10, -1, 10]",94.199997,20.700001,2.0,0.1,0.7,9.000000,29.000000,0.5,6.400000,96.000000,125.699997,0.0,0.0,0.5,5.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,닭고기시금치그라탕,"[닭고기, 가슴살, 날것, 시금치, 생것, 노지, 당근, 생것, 양파, 생것, 국내...","[27, 62, 28, 49, 7, 50, 47, 7, 66, 7, 31, 98, ...","[-1, -1, 30, -1, -1, 10, -1, 10, -1, -1, 10, -...",117.599998,4.200000,11.7,6.0,0.8,133.199997,256.600006,0.6,265.100006,223.399994,241.699997,0.1,0.2,3.5,7.9
144,닭죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 닭고기, 가슴살, 날것, 분유,...","[0, 1, 2, 3, 4, 5, 27, 62, 28, 63, 64]","[-1, -1, -1, -1, -1, 20, -1, -1, 20, -1, 15]",162.000000,23.700001,8.8,3.2,0.2,97.599998,123.099998,1.4,35.200001,144.300003,87.599998,0.1,0.2,3.1,7.0
145,도미살무른밥,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 돔, 감성돔, 배추, 생것, 양...","[0, 1, 2, 3, 4, 5, 83, 84, 40, 7, 73, 7, 52, 53]","[-1, -1, -1, -1, -1, 30, -1, 30, -1, 10, -1, 1...",155.000000,27.500000,9.2,0.7,3.3,89.000000,128.800003,1.4,338.700012,982.599976,15.600000,0.1,0.2,2.2,5.2
146,달걀콩닭고기수프,"[계란, 전란, 생것, 닭고기, 가슴살, 날것, 당근, 생것, 완두콩, 생것, 옥수...","[41, 61, 7, 27, 62, 28, 47, 7, 17, 7, 15, 16, ...","[-1, -1, 20, -1, -1, 20, -1, 10, -1, 10, -1, -...",129.899994,11.200000,10.5,4.6,1.5,84.300003,145.100006,1.6,55.900002,228.100006,221.600006,0.2,0.2,7.5,5.7


In [15]:
# 데이터 전처리

# ingredient
data_ingredient = df.iloc[:,:4]
data_ingredient # 재료들만, 영양소&칼로리 제외

# training_data_set
tds = data_ingredient.iloc[:,2:]
tds

# ingredient_index와 ingredient_gram을 문자열로 변환하여 하나의 열로 합침
tds['ingredient_combined'] = tds.apply(lambda row: ' '.join(map(str, row['ingredient_index'] + row['ingredient_gram'])), axis=1)

# CountVectorizer로 특징 벡터 생성
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(tds['ingredient_combined'])

/var/tmp/ipykernel_379535/2401688122.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tds['ingredient_combined'] = tds.apply(lambda row: ' '.join(map(str, row['ingredient_index'] + row['ingredient_gram'])), axis=1)


In [17]:
# KMeans 군집화 모델 생성
num_clusters = 3  # 군집 개수 설정
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(X)

/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KMeans(n_clusters=3, random_state=42)

In [18]:
# 군집 결과를 데이터프레임에 추가
df['cluster'] = kmeans.labels_
df

,food_name,ingredient_name,ingredient_index,ingredient_gram,calorie,carbohydrate,protein,fat,fiber,calcium,phosphorus,iron,sodium,potassium,vitamin_A,thiamine,riboflavin,niacin,vitamin_C,cluster
0,찹쌀미음,"[쌀, 찹쌀, 백미]","[0, 18, 19]","[-1, -1, 23]",86.000000,18.700001,1.7,0.1,0.1,0.900000,34.700001,0.5,0.700000,43.900002,0.000000,0.0,0.0,0.4,0.0,2
1,닭가슴살고구마죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 닭고기, 가슴살, 날것, 고구마...","[0, 1, 2, 3, 4, 5, 27, 62, 28, 8, 7]","[-1, -1, -1, -1, -1, 20, -1, -1, 20, -1, 20]",117.000000,21.400000,6.2,0.4,0.7,9.800000,63.400002,0.5,16.400000,157.199997,4.800000,0.0,0.0,2.7,5.2,1
2,닭가슴살당근죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 닭고기, 가슴살, 날것, 당근,...","[0, 1, 2, 3, 4, 5, 27, 62, 28, 47, 7]","[-1, -1, -1, -1, -1, 20, -1, -1, 20, -1, 20]",98.199997,16.900000,6.1,0.4,0.8,12.600000,60.000000,0.5,19.200001,143.800003,252.399994,0.0,0.0,2.7,1.4,1
3,닭가슴살청경채죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 닭고기, 가슴살, 날것, 청경채...","[0, 1, 2, 3, 4, 5, 27, 62, 28, 39]","[-1, -1, -1, -1, -1, 20, -1, -1, 20, 15]",93.500000,15.700000,6.1,0.4,0.7,18.500000,58.299999,0.5,15.600000,105.400002,52.799999,0.0,0.0,2.6,7.4,1
4,찹쌀컬리플라워당근죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 쌀, 찹쌀, 백미, 컬리플라워,...","[0, 1, 2, 3, 4, 5, 0, 18, 19, 67, 47, 7]","[-1, -1, -1, -1, -1, 20, -1, -1, 5, 10, -1, 10]",94.199997,20.700001,2.0,0.1,0.7,9.000000,29.000000,0.5,6.400000,96.000000,125.699997,0.0,0.0,0.5,5.4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,닭고기시금치그라탕,"[닭고기, 가슴살, 날것, 시금치, 생것, 노지, 당근, 생것, 양파, 생것, 국내...","[27, 62, 28, 49, 7, 50, 47, 7, 66, 7, 31, 98, ...","[-1, -1, 30, -1, -1, 10, -1, 10, -1, -1, 10, -...",117.599998,4.200000,11.7,6.0,0.8,133.199997,256.600006,0.6,265.100006,223.399994,241.699997,0.1,0.2,3.5,7.9,0
144,닭죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 닭고기, 가슴살, 날것, 분유,...","[0, 1, 2, 3, 4, 5, 27, 62, 28, 63, 64]","[-1, -1, -1, -1, -1, 20, -1, -1, 20, -1, 15]",162.000000,23.700001,8.8,3.2,0.2,97.599998,123.099998,1.4,35.200001,144.300003,87.599998,0.1,0.2,3.1,7.0,1
145,도미살무른밥,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 돔, 감성돔, 배추, 생것, 양...","[0, 1, 2, 3, 4, 5, 83, 84, 40, 7, 73, 7, 52, 53]","[-1, -1, -1, -1, -1, 30, -1, 30, -1, 10, -1, 1...",155.000000,27.500000,9.2,0.7,3.3,89.000000,128.800003,1.4,338.700012,982.599976,15.600000,0.1,0.2,2.2,5.2,0
146,달걀콩닭고기수프,"[계란, 전란, 생것, 닭고기, 가슴살, 날것, 당근, 생것, 완두콩, 생것, 옥수...","[41, 61, 7, 27, 62, 28, 47, 7, 17, 7, 15, 16, ...","[-1, -1, 20, -1, -1, 20, -1, 10, -1, 10, -1, -...",129.899994,11.200000,10.5,4.6,1.5,84.300003,145.100006,1.6,55.900002,228.100006,221.600006,0.2,0.2,7.5,5.7,0


In [19]:
# 군집별 대표 재료 찾기
cluster_centers = kmeans.cluster_centers_
top_words = vectorizer.get_feature_names_out()
for i, center in enumerate(cluster_centers):
    top_indices = center.argsort()[-5:]  # 각 군집에서 가장 높은 빈도의 단어 인덱스 선택
    top_ingredients = [top_words[idx] for idx in top_indices]
    print(f"Cluster {i+1} Top Ingredients: {', '.join(top_ingredients)}")

Cluster 1 Top Ingredients: 26, 65, 41, 46, 17
Cluster 2 Top Ingredients: 21, 20, 12, 27, 25
Cluster 3 Top Ingredients: 22, 15, 17, 10, 35


In [20]:
cluster1 = df[df['cluster'] == 1]
cluster1

,food_name,ingredient_name,ingredient_index,ingredient_gram,calorie,carbohydrate,protein,fat,fiber,calcium,phosphorus,iron,sodium,potassium,vitamin_A,thiamine,riboflavin,niacin,vitamin_C,cluster
1,닭가슴살고구마죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 닭고기, 가슴살, 날것, 고구마...","[0, 1, 2, 3, 4, 5, 27, 62, 28, 8, 7]","[-1, -1, -1, -1, -1, 20, -1, -1, 20, -1, 20]",117.000000,21.400000,6.2,0.4,0.7,9.800000,63.400002,0.5,16.400000,157.199997,4.800000,0.0,0.0,2.7,5.2,1
2,닭가슴살당근죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 닭고기, 가슴살, 날것, 당근,...","[0, 1, 2, 3, 4, 5, 27, 62, 28, 47, 7]","[-1, -1, -1, -1, -1, 20, -1, -1, 20, -1, 20]",98.199997,16.900000,6.1,0.4,0.8,12.600000,60.000000,0.5,19.200001,143.800003,252.399994,0.0,0.0,2.7,1.4,1
3,닭가슴살청경채죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 닭고기, 가슴살, 날것, 청경채...","[0, 1, 2, 3, 4, 5, 27, 62, 28, 39]","[-1, -1, -1, -1, -1, 20, -1, -1, 20, 15]",93.500000,15.700000,6.1,0.4,0.7,18.500000,58.299999,0.5,15.600000,105.400002,52.799999,0.0,0.0,2.6,7.4,1
7,애호박완두콩무른밥,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 호박, 애호박, 생것, 완두콩,...","[0, 1, 2, 3, 4, 5, 13, 14, 7, 17, 7]","[-1, -1, -1, -1, -1, 25, -1, -1, 20, -1, 5]",100.599998,22.000000,2.3,0.2,0.8,9.900000,28.200001,0.5,1.000000,98.199997,8.000000,0.1,0.0,2.7,7.0,1
17,단호박죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 호박, 당호박]","[0, 1, 2, 3, 4, 5, 13, 24]","[-1, -1, -1, -1, -1, 20, -1, 20]",75.400002,16.700001,1.7,0.1,0.6,7.200000,20.600000,0.3,1.000000,76.599998,38.200001,0.0,0.0,0.5,3.8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,쇠고기양송이무른밥,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 쇠고기, 한우, 사태, 양송이,...","[0, 1, 2, 3, 4, 5, 21, 22, 23, 73, 7]","[-1, -1, -1, -1, -1, 20, -1, -1, 20, -1, 20]",100.000000,15.700000,7.0,12.8,0.5,5.600000,71.400002,0.9,12.200000,177.000000,2.400000,0.1,0.1,2.2,1.0,1
135,감자수제비,"[쌀, 멥쌀, 논벼, 쌀가루, 밀가공식품, 밀가루, 중력분, 감자, 생것, 호박, ...","[0, 1, 2, 94, 95, 96, 97, 6, 7, 13, 14, 7, 41,...","[-1, -1, -1, 10, -1, -1, 10, -1, 10, -1, -1, 3...",127.400002,21.500000,5.1,2.6,2.2,53.400002,74.699997,0.9,181.600006,546.799988,44.099998,0.1,0.1,0.7,13.5,1
136,쇠고기청경채죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 쇠고기, 한우, 사태, 청경채 ...","[0, 1, 2, 3, 4, 5, 21, 22, 23, 39]","[-1, -1, -1, -1, -1, 20, -1, -1, 20, 20]",99.400002,15.800000,6.5,12.8,0.8,22.400000,61.400002,0.9,14.600000,112.400002,71.400002,0.0,0.1,1.5,10.0,1
140,쇠고기난자완스,"[쇠고기, 한우, 사태, 양파, 생것, 국내산, 계란, 난황, 생것, 전분, 감자녹...","[21, 22, 23, 66, 7, 31, 41, 42, 7, 43, 44, 9, 7]","[-1, -1, 10, -1, -1, 15, -1, -1, 20, -1, 10, -...",124.000000,10.100000,6.5,12.4,0.7,40.599998,168.600006,1.8,17.200001,112.900002,111.000000,0.1,0.1,0.7,16.1,1


In [21]:
cluster1 = df[df['cluster'] == 0]
cluster1

,food_name,ingredient_name,ingredient_index,ingredient_gram,calorie,carbohydrate,protein,fat,fiber,calcium,phosphorus,iron,sodium,potassium,vitamin_A,thiamine,riboflavin,niacin,vitamin_C,cluster
4,찹쌀컬리플라워당근죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 쌀, 찹쌀, 백미, 컬리플라워,...","[0, 1, 2, 3, 4, 5, 0, 18, 19, 67, 47, 7]","[-1, -1, -1, -1, -1, 20, -1, -1, 5, 10, -1, 10]",94.199997,20.700001,2.0,0.1,0.7,9.000000,29.000000,0.5,6.400000,96.000000,125.699997,0.0,0.0,0.5,5.400000,0
5,고구마찹쌀진밥,"[쌀, 찹쌀, 백미, 고구마, 생것]","[0, 18, 19, 8, 7]","[-1, -1, 20, -1, 10]",87.599998,19.299999,1.6,0.1,0.4,3.200000,35.599998,0.5,2.100000,81.099998,1.900000,0.0,0.0,0.4,2.500000,0
9,브로콜리당근죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 브로콜리, 생것, 당근, 생것]","[0, 1, 2, 3, 4, 5, 9, 7, 47, 7]","[-1, -1, -1, -1, -1, 20, -1, 10, -1, 10]",75.800003,16.500000,1.9,0.1,0.8,13.000000,36.599998,0.5,4.300000,87.300003,138.500000,0.0,0.0,0.5,10.400000,0
10,표고버섯무른밥,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 표고버섯, 생것, 양파, 생것,...","[0, 1, 2, 3, 4, 5, 77, 7, 66, 7, 31]","[-1, -1, -1, -1, -1, 30, -1, 10, -1, -1, 10]",111.699997,24.400000,2.3,0.2,1.0,6.100000,26.000000,0.5,1.500000,70.300003,0.000000,0.0,0.0,0.8,1.300000,0
28,고구마야채무른밥,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 고구마, 생것, 양배추, 생것,...","[0, 1, 2, 3, 4, 5, 8, 7, 36, 7, 47, 7, 9, 7]","[-1, -1, -1, -1, -1, 25, -1, 5, -1, 10, -1, 10...",102.699997,22.600000,2.5,0.2,1.2,18.700001,45.200001,0.6,5.600000,136.000000,139.800003,0.0,0.0,0.6,14.600000,0
80,브로콜리양송이죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 브로콜리, 생것, 양송이, 생것]","[0, 1, 2, 3, 4, 5, 9, 7, 73, 7]","[-1, -1, -1, -1, -1, 20, -1, 10, -1, 10]",74.099998,15.900000,2.2,0.1,0.6,9.800000,41.700001,0.5,1.700000,106.099998,12.800000,0.0,0.1,0.8,10.100000,0
81,닭고기완자핑거푸드,"[닭고기, 날것, 양파, 생것, 국내산, 우엉, 생것, 당근, 생것]","[27, 28, 66, 7, 31, 82, 7, 47, 7]","[-1, 30, -1, -1, 10, -1, 10, -1, 10]",65.000000,3.000000,6.1,3.2,0.9,14.200000,64.599998,0.5,21.000000,141.300003,142.199997,0.0,0.1,2.3,2.300000,0
94,치즈야채무른밥,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 당근, 생것, 호박, 애호박, ...","[0, 1, 2, 3, 4, 5, 47, 7, 13, 14, 7, 66, 7, 31...","[-1, -1, -1, -1, -1, 30, -1, 10, -1, -1, 10, -...",161.899994,26.400000,5.0,3.8,0.8,86.900002,157.399994,0.6,174.300003,121.699997,164.100006,0.1,0.1,0.6,4.900000,0
97,감자야채달걀노른자찜,"[감자, 생것, 배추, 생것, 계란, 난황, 생것]","[6, 7, 40, 7, 41, 42, 7]","[-1, 10, -1, 10, -1, -1, 20]",74.500000,1.600000,3.5,6.0,0.3,30.200001,119.599998,1.2,9.900000,79.800003,91.199997,0.1,0.1,0.2,4.900000,0
115,닭가슴살요구르트샐러드,"[닭고기, 가슴살, 날것, 양배추, 생것, 감자, 생것, 당근, 생것, 요구르트, 호상]","[27, 62, 28, 36, 7, 6, 7, 47, 7, 108, 109]","[-1, -1, 30, -1, 10, -1, 10, -1, 10, -1, 10]",55.000000,4.400000,7.8,0.6,0.7,28.100000,82.500000,0.4,23.500000,174.500000,130.399994,0.1,0.1,3.6,6.700000,0


In [22]:
cluster1 = df[df['cluster'] == 2]
cluster1

,food_name,ingredient_name,ingredient_index,ingredient_gram,calorie,carbohydrate,protein,fat,fiber,calcium,phosphorus,iron,sodium,potassium,vitamin_A,thiamine,riboflavin,niacin,vitamin_C,cluster
0,찹쌀미음,"[쌀, 찹쌀, 백미]","[0, 18, 19]","[-1, -1, 23]",86.000000,18.700001,1.7,0.1,0.1,0.900000,34.700001,0.5,0.700000,43.900002,0.000000,0.0,0.0,0.4,0.000000,2
6,멜론감자수프,"[멜론, 머스크, 감자, 생것]","[85, 86, 6, 7]","[-1, 50, -1, 30]",35.500000,8.000000,1.4,0.1,0.9,5.300000,36.799999,0.5,7.400000,305.799988,1.500000,0.1,0.0,0.8,17.299999,2
8,감자바나나요구르트으깸,"[감자, 생것, 바나나, 생과, 요구르트, 호상]","[6, 7, 29, 11, 108, 109]","[-1, 30, -1, 30, -1, 10]",54.700001,12.400000,1.5,0.3,1.0,20.500000,33.900002,0.5,1.800000,219.300003,5.000000,0.0,0.0,0.6,9.500000,2
11,콩나물무른밥,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 콩나물, 생것]","[0, 1, 2, 3, 4, 5, 92, 7]","[-1, -1, -1, -1, -1, 30, -1, 20]",110.400002,23.400000,3.0,0.4,0.8,10.400000,32.500000,0.5,1.200000,74.000000,0.000000,0.0,0.0,0.6,1.600000,2
12,두부닭죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 쌀, 찹쌀, 백미, 닭고기, 가...","[0, 1, 2, 3, 4, 5, 0, 18, 19, 27, 62, 28, 46, 46]","[-1, -1, -1, -1, -1, 15, -1, -1, 5, -1, -1, 20...",108.500000,16.200001,7.7,1.0,0.7,36.299999,74.800003,1.0,15.200000,76.000000,1.000000,0.0,0.0,2.7,0.200000,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,감자완자핑거푸드,"[감자, 생것, 쇠고기, 한우, 사태, 당근, 생것, 브로콜리, 생것]","[6, 7, 21, 22, 23, 47, 7, 9, 7]","[-1, 30, -1, -1, 10, -1, 15, -1, 15]",39.299999,5.200000,4.1,6.4,1.3,17.900000,70.300003,0.8,12.400000,242.399994,209.000000,0.1,0.1,1.2,22.100000,2
134,흑미버섯무른밥,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 쌀, 흑미, 멥쌀, 송이버섯, 생것]","[0, 1, 2, 3, 4, 5, 0, 20, 1, 65, 7]","[-1, -1, -1, -1, -1, 25, -1, -1, 5, -1, 30]",113.400002,24.600000,2.8,0.3,1.8,5.800000,42.599998,0.9,1.500000,131.199997,0.000000,0.1,0.1,2.0,0.000000,2
137,연두부아욱진밥,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 두부, 연두부, 아욱]","[0, 1, 2, 3, 4, 5, 46, 51, 68]","[-1, -1, -1, -1, -1, 30, -1, 30, 25]",121.699997,23.700001,4.4,1.0,1.5,46.299999,58.799999,1.3,10.800000,195.600006,285.799988,0.1,0.1,0.7,12.000000,2
138,밥경단,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 쇠고기, 한우, 사태, 표고버섯...","[0, 1, 2, 3, 4, 5, 21, 22, 23, 77, 7, 68]","[-1, -1, -1, -1, -1, 30, -1, -1, 20, -1, 15, 15]",140.100006,24.100000,7.7,12.9,1.8,20.500000,74.199997,1.3,17.600000,197.500000,173.800003,0.1,0.1,2.2,8.400000,2


In [ ]:
# 결론 
# 잘 안 된 것 같음ㅠ